# Прогнозирование стоимости автомобиля по характеристикам


Образовательная платформа: SkillFactory

Специализация: Data Science

Группа: DST-37 и 38

Юнит 6. Проект 5: "Выбираем автомобиль правильно"


### Задача:

    Создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам для того, чтобы выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены).

### Метрика:

    MAPE (Mean Percentage Absolute Error) - средняя абсолютная ошибка в процентах

### Нужно:

    Составить train датасет - спарсить данные, либо найти готовый
    Обучить модель

### Плюс:

    Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
    Сгенерировать новые признаки
    Подгрузить еще больше данных
    Попробовать подобрать параметры модели
    Попробовать разные алгоритмы и библиотеки ML
    Сделать Ансамбль моделей, Blending, Stacking

### Этапы работы:

    Парсинг с авто.ру - Вадим, Евгений, Артём
    EDA, Feature Engineering - Артём, Вадим, Евгений
    Сравнение одиночных моделей - Артём, Вадим
    Стекинг - Вадим
    
    
#### В данном ноутбуке мы выполняем подготовку данных.

#### Также в этом проекте мы использовали:

Ноутбук, через который пытались парсить: https://www.kaggle.com/artemskakun/carpriceprediction-autoruparser

Спарсенный датасет мы взяли у этой команды, потому что парсинг занимал очень много времени: https://www.kaggle.com/juliadeinego/data-car-sales

Ноутбук, в котором провели ML: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-iron-fingers

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime
from ast import literal_eval

import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns', None)

RANDOM_SEED = 42

AUTORU_DATA = '../input/data-parsing'
TEST_DATA = '../input/sf-dst-car-price-prediction/'

In [2]:
# Определение функций
def normalise_doors(row):
    num = row['number_of_doors']
    if pd.isna(num) or num == 0:
        result = 4
        vehicle = row['body_type']
        if vehicle == 'минивэн':
            result = 5
        elif vehicle == 'седан':
            result = 4
        elif vehicle == 'купе':
            result = 2
        return result
    return num

def normalise_model_name(row):
    name = ''
    try:
        d = literal_eval(row['equipment_dict'])
        name = d['model']
    except Exception as e: 
        try:
            val = row['model_name'].split('|')
            if len(val)>1:
                name = val[0]
        except Exception as e2:
            name = row['model_name']
    return name


def normalise_brand_name(row):
    name = ''
    try:
        d = literal_eval(row['equipment_dict'])
        name = d['mark']
    except Exception as e: 
        name = row['brand']
    return name

def normalise_model_date(row):
    val = ''
    try:
        d = literal_eval(row['equipment_dict'])
        val = d['year']
    except Exception as e:
        val = row['production_date']
    finally: #but this work
        try:
            i = int(val)
            return i
        except Exception as e2:
            return 0
        
def normalise_electric_engine(df, row):
    power = row['engine_power']
    mean = df[(df['fuel_type']!='электро') \
                 & (df['engine_power']>=power*0.8) \
                 & (df['engine_power']<=power*1.2)].engine_displacement.mean()
    return int(round(mean, -3))
    
    
rates = {
    '20210416' : 76.9808,
    '20210417' : 75.5535,
    '20210511' : 74.1373,
    '20201021' : 77.7780,
    '20210415' : 75.6826,
    '20201020' : 77.9241,
    '20201019' : 77.9644,
    '20201025' : 76.4667,
    '20201024' : 76.4667,
    '20201026' : 76.4667
}

def price_exchange(row, column, exchange_to='d'):
    rate = rates.get(row['parcing_date'])
    if exchange_to=='d':
        return int(row[column] / rate)
    return int(row[column] * rate)

def remove_whitespace(x):
    try:
        x = "".join(x.split())
    except:
        pass
    return x

def count_words(string, good_words, bad_words):
    i = 0
    try:
        for word in string:
            if type(word) == float:
                continue
            try:
                if word in good_words:
                    i += 1
                elif word in bad_words:
                    i -= 1
            except:
                print(word)
    except:
        pass
    return(i)

### Считываем все файлы и объединяем в единый датафрейм

In [3]:
print('List of data files in directory "data" ->',glob.glob('../input/data-parsing'))

required_columns = ['sell_id','parsing_unixtime','price','bodyType','color','engineDisplacement','enginePower',
                    'fuelType','mileage','productionDate','vehicleTransmission','Владельцы','ПТС','Руль',
                    'Состояние','brand','model_name','equipment_dict','description']

full_df = pd.DataFrame()
for file in glob.glob('../input/data-parsing/*.csv'):
    file_name = file[file.rindex('/')+1:]
    print(file_name)
    df = pd.DataFrame()
    try:
        df = pd.read_csv(file)
    except:
        df = pd.read_csv(file, delimiter=';')
    df = df[required_columns]
    full_df = pd.concat([full_df, df], ignore_index=True)
    
full_df.sample(2)

List of data files in directory "data" -> []
bmw.csv
nissan.csv
infiniti.csv
honda.csv
audi.csv
mitsubishi.csv
volkswagen.csv


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


toyota.csv


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


volvo.csv
lexus.csv
skoda.csv
mercedes.csv


,sell_id,parsing_unixtime,price,bodyType,color,engineDisplacement,enginePower,fuelType,mileage,productionDate,vehicleTransmission,Владельцы,ПТС,Руль,Состояние,brand,model_name,equipment_dict,description
79561,№ 1102391439,1620771969.0,435 000 ₽,седан,белый,1.6 л,105 л.с.,Бензин,159 000 км,2014,механическая,2 владельца,Оригинал,Левый,Не требует ремонта,Volkswagen,Polo,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",Торг у капота
103198,№ 1076442800,1618591182.0,2 250 000 ₽,внедорожник 5 дв.,чёрный,4.7 л,288 л.с.,Бензин,240 000 км,2008,автоматическая,2 владельца,Оригинал,Левый,Не требует ремонта,Toyota,Land Cruiser,"{'asciiCat': 'cars', 'category': 'cars', 'engi...","Машина на отличном ходу, все работает исправно..."



### Удалим дубликаты и пустые строки из спарсенных данных

In [5]:
full_df = full_df.drop_duplicates(subset=['sell_id'])
full_df = full_df.dropna(how='all')
len(full_df)

101304

### Оставляем только необходимые признаки

In [6]:
# создаем список с новым названием колонок
new_columns = ['sell_id','parsing_unixtime','price','body_type','color','engine_displacement','engine_power',
               'fuel_type','mileage','production_date','vehicle_transmission','owners','vehicle_pasport',
               'wheel','condition','brand','model_name','equipment_dict','description']
        
# оставляем в датасете выбранные колонки
full_df = full_df[required_columns]

# переименуем колонки
full_df.columns = new_columns

full_df.sample(2)

,sell_id,parsing_unixtime,price,body_type,color,engine_displacement,engine_power,fuel_type,mileage,production_date,vehicle_transmission,owners,vehicle_pasport,wheel,condition,brand,model_name,equipment_dict,description
59731,№ 1103139489,1618583409.0,800 000 ₽,внедорожник 5 дв.,чёрный,3.2 л,160 л.с.,Дизель,303 000 км,2005,механическая,2 владельца,Оригинал,Левый,Не требует ремонта,Mitsubishi,Pajero,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",Продаю отличный автомобиль! Я второй хозяин. С...
21596,№ 1102851906,1618643332.0,149 000 ₽,седан,синий,2.5 л,210 л.с.,Бензин,250 000 км,2001,автоматическая,3 или более,Дубликат,Правый,Не требует ремонта,Nissan,Cefiro,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",Легенда бизнес класса тех времен.Чистая япошка...


### Проведём первичную обработку датасета

In [7]:
def preprocessing_autoru_df(df):
    df.drop(['sell_id'], axis=1, inplace=True)
    # model date
    df['production_date'] = df['production_date'].fillna(0)
    df['model_date'] = df.apply(lambda row : normalise_model_date(row), axis=1)
    df['model_name'] = df.apply(lambda row : normalise_model_name(row), axis=1)
    df['brand'] = df.apply(lambda row : normalise_brand_name(row), axis=1)
    df.drop(df[(df['model_name'].isna()) & (full_df['brand'].isna())].index, inplace=True)
    
    df.drop('equipment_dict', axis=1, inplace=True)

    df['mileage'] = df['mileage'].fillna('0')
    df['mileage'] = df['mileage'].apply(remove_whitespace)
    df['mileage'] = df['mileage'].str.extract('(\d+)', expand=True)
    df['mileage'] = df['mileage'].fillna(0).astype(int)

    
preprocessing_autoru_df(full_df)
full_df.head(2)

,parsing_unixtime,price,body_type,color,engine_displacement,engine_power,fuel_type,mileage,production_date,vehicle_transmission,owners,vehicle_pasport,wheel,condition,brand,model_name,description,model_date
0,1618565279.0,95 000 ₽,седан,пурпурный,1.8 л,113 л.с.,Бензин,385000,1992,механическая,3 или более,Дубликат,Левый,Не требует ремонта,BMW,3ER,В хорошем состоянии. Двигатель в отличном сост...,1992
1,1618565279.0,7 990 000 ₽,купе,серый,1.5 л,231 л.с.,Гибрид,17979,2018,автоматическая,3 или более,Оригинал,Левый,Не требует ремонта,BMW,I8,"В продаже автомобиль от компании АО ABTODOM, о...",2018


### Загружаем тестовый датасет

In [8]:
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
test.sample(2)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
1284,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,зелёный,NaN,Всем добрый день!\nПродаю свой любимый авто. \...,1.8 LTR,150 N12,"{""engine-proof"":true,""tinted-glass"":true,""esp""...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,301329,1996,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.8 AT (150 л.с.),5,1603231116,RUB,2000,1101127476,"{""id"":""20501027"",""displacement"":1781,""engine_t...",LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,3 или более,1 год и 1 месяц,Оригинал,передний,Левый,Не требует ремонта,Растаможен
32728,хэтчбек 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/march/10...,серебристый,NaN,Отличный автомобиль. Ездила жена в пределах ра...,1.2 LTR,90 N12,"{""tinted-glass"":true,""airbag-driver"":true,""aux...",бензин,https://autoru.naydex.net/1xsMU7627/eea2057RFg...,122000,2002,"{""code"":""MARCH"",""name"":""March"",""ru_name"":""Марч...",MARCH,1.2 AT (90 л.с.),5,1603605574,RUB,2004,1093060658,"{""id"":""8299524"",""displacement"":1240,""engine_ty...",HATCHBACK_5_DOORS AUTOMATIC 1.2,автоматическая,JAPANESE,3 или более,1 год и 7 месяцев,Оригинал,передний,Правый,Не требует ремонта,Растаможен


In [9]:
# выберем интересующие нас признаки
columns = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower',
           'fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 'parsing_unixtime',
           'productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Руль', 'Состояние', 'description']

df_test = test[columns]

# создаем список с новым названием признаков
new_columns = ['body_type', 'brand', 'color', 'engine_displacement', 'engine_power',
               'fuel_type', 'mileage', 'model_date', 'model_name', 'number_of_doors', 'parsing_unixtime',
               'production_date', 'vehicle_transmission', 'owners', 'vehicle_pasport',
               'wheel', 'condition', 'description']

df_test.columns = new_columns

df_test.sample(2)

,body_type,brand,color,engine_displacement,engine_power,fuel_type,mileage,model_date,model_name,number_of_doors,parsing_unixtime,production_date,vehicle_transmission,owners,vehicle_pasport,wheel,condition,description
355,лифтбек,SKODA,серый,1.8 LTR,152 N12,бензин,141532,2008,OCTAVIA,5,1603227649,2009,роботизированная,2 владельца,Оригинал,Левый,Не требует ремонта,",\nБезопасная покупка в АСЦ/ЧЕСТНО –легко! \nС..."
7888,седан,BMW,чёрный,3.0 LTR,249 N12,дизель,7340,2019,7ER,4,1603108977,2019,автоматическая,1 владелец,Оригинал,Левый,Не требует ремонта,«Inchcape Certified- Первый международный офи...


### Объединим оба датасета и проведем дальнейшую обработку одновременно

In [10]:
full_df['test'] = 0
df_test['test'] = 1
df_test['price'] = 0
full_df = pd.concat([full_df, df_test], ignore_index=True)

<ipython-input-10-029da23d6a0b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['test'] = 1
<ipython-input-10-029da23d6a0b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['price'] = 0


### Проведем дальнейшую предобработку для объединенного датафрейма

In [11]:
# найдём и удалим строку с неверными значениями
a = full_df[full_df['parsing_unixtime'] == 'https://avatars.mds.yandex.net/get-autoru-vos/2090686/56351fff854ba117076587162cfc6805/1200x900n'].index
full_df = full_df.drop(a, axis=0)

def preprocessing_full_df(df):
    # убираем пустые строки
    df.dropna(axis=1, how='all', inplace=True)
    
    # обрабатываем признак engine_power 
    df['engine_power'] = df['engine_power'].astype(str).str.extract('(\d+)', expand=True)\
        .fillna(0).astype(int)
    
    # обрабатываем признак engine_displacement
    df['engine_displacement'] = ((df['engine_displacement']).astype(str)\
        .str.extract('([0-9.]+)', expand=True).astype(float)\
        .fillna(0)*1000).astype(int)
    df['engine_displacement'] = df.apply(lambda row : normalise_electric_engine(df, row) \
        if (row.fuel_type=='электро') | (row.engine_displacement==0) else row.engine_displacement , axis=1)
    
    # обрабатываем признак number_of_doors
    df['number_of_doors'] = df.apply(lambda row : normalise_doors(row), axis=1).astype(int).apply(str)

    # обрабатываем признак body_type
    df['body_type'] = df['body_type'].fillna('седан')\
        .astype(str).apply(lambda x: x[:x.find(" ")] if x.find(" ") > 0 else x)
    
    # обрабатываем признак fuel_type
    df['fuel_type'] = df['fuel_type'].fillna('бензин').str.strip()\
        .replace({"бензин, газобаллонное оборудование": "бго", 
                  "газ, газобаллонное оборудование": "газ",
                  "дизель, газобаллонное оборудование": "дизель",
                  "гибрид, газобаллонное оборудование": "гибрид"}, inplace=True)
    
    # обрабатываем признак owners
    df['owners'] = df['owners'].astype(str)\
        .str.extract('(\d+)', expand=True)\
        .fillna('1').astype(int)

    # обрабатываем признак price
    df['price'] = df['price'].apply(remove_whitespace)\
        .str.extract('(\d+)', expand=True)\
        .fillna(0).astype(int)

    # обрабатываем признак vehicle_pasport
    df['vehicle_pasport'] = df['vehicle_pasport'].fillna('Оригинал')

    # обрабатываем признак production_date
    df['production_date'] = df['production_date'].fillna(df['production_date'].median()).astype(int)
    df['model_date'] = df['model_date'].fillna(value=df['production_date']).astype(int)
    
    # переводим признаки в нижний регистр
    for col in ['body_type','color','fuel_type','vehicle_transmission','vehicle_pasport',
                'wheel','brand']:
        df[col] = df[col].astype(str).str.lower()
    
    # переводим model_name в верхний регистр
    df['model_name'] = df['model_name'].astype(str).str.upper()

    # удаляем лишние признаки
    df.drop(df[df['condition'] == 'Битый / не на ходу'].index, inplace=True)
    df.drop('parsing_unixtime', axis=1, inplace=True)
    df.drop('production_date', axis=1, inplace=True)

preprocessing_full_df(full_df)
full_df.sample(2)

,price,body_type,color,engine_displacement,engine_power,fuel_type,mileage,vehicle_transmission,owners,vehicle_pasport,wheel,condition,brand,model_name,description,model_date,test,number_of_doors
70141,1495000,внедорожник,чёрный,2000,146,none,83000,вариатор,2,оригинал,левый,Не требует ремонта,toyota,RAV_4,Японская сборка. Дата первой продажи из салона...,2013,0,4
52561,570000,компактвэн,чёрный,1200,105,none,191000,механическая,3,оригинал,левый,Не требует ремонта,volkswagen,CADDY,Volkswagen Caddy в хорошем техническом состоян...,2012,0,4


In [12]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135986 entries, 0 to 135988
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   price                 135986 non-null  int64 
 1   body_type             135986 non-null  object
 2   color                 135986 non-null  object
 3   engine_displacement   135986 non-null  int64 
 4   engine_power          135986 non-null  int64 
 5   fuel_type             135986 non-null  object
 6   mileage               135986 non-null  int64 
 7   vehicle_transmission  135986 non-null  object
 8   owners                135986 non-null  int64 
 9   vehicle_pasport       135986 non-null  object
 10  wheel                 135986 non-null  object
 11  condition             135986 non-null  object
 12  brand                 135986 non-null  object
 13  model_name            135986 non-null  object
 14  description           135433 non-null  object
 15  model_date       

### Обработаем текстовое поле description

Попытаемся получить дополнительный признак о наличии положительных и отрицательных слов в тексте объявления

In [13]:
words_dct = {}
for description in full_df['description']:
    if type(description) == float:
        continue
    description = description.lower()
    description = description.replace('!', '').replace('.', '').replace(',', '')
    description = description.split(' ')
    for word in description:
        if word in words_dct:
            words_dct[word] += 1
        else:
            words_dct[word] = 1
words_dct

{'в': 363219,
 'хорошем': 18042,
 'состоянии': 45292,
 'двигатель': 13122,
 'отличном': 22943,
 'состояние': 17292,
 'сухой': 237,
 'новая': 8267,
 'помпа': 821,
 'грм': 3455,
 'передние': 19011,
 'рычаги': 1153,
 'тормозные': 4667,
 'диски': 12578,
 'резина': 17587,
 'зима': 6655,
 'кожаный': 7618,
 'салон': 23583,
 'музыка': 3922,
 'стеклоподъёмники': 252,
 'машина': 29492,
 'не': 154626,
 'гнилая': 314,
 'стаканы': 87,
 'родные': 2618,
 'варились': 9,
 'и': 300956,
 'красилась': 463,
 'движок': 526,
 'родной': 10725,
 'машины': 4490,
 'полностью': 13753,
 'на': 212130,
 'ходу': 3399,
 'вложений': 11062,
 'требует': 10379,
 'юридический': 55,
 'все': 67906,
 'чист': 699,
 'торг': 24975,
 'обмен': 19293,
 'автокредит': 2250,
 'наша': 207,
 'компания': 1370,
 'amirravto': 20,
 'уже': 4850,
 'более': 23493,
 '5': 7639,
 'лет': 12165,
 'успешно': 680,
 'работает': 13086,
 'автомобильном': 589,
 'рынке': 2092,
 'за': 33120,
 'это': 11592,
 'время': 12351,
 'динамично': 23,
 'развивается':

In [14]:
# сделаем два списка: положительные и отрицательные слова
good_words = ['комплект', 'официального', 'салон', 'птс', 'отличном', 'выгода', 'гарантия',
             'комплексную', 'оригинал', 'гарантию', 'официальный', 'подарок', 'гарантии',
              'новая', 'сертифицированными', '✅', 'предпродажную', 'новые', 'favorit',
              'отличное', 'надежный', 'официальным', 'отличный', 'ремонтировалась', 'хорошем']
bad_words = ['дтп', 'срочный', 'трейд-ин', 'ремонтов', 'ремонта', 'капремонта',
             'ремонтных', 'дтпвсе', 'авария', 'аварии']

# заполним поле description цифровыми данными
full_df['description'] = full_df['description'].apply(lambda x: count_words(x, good_words, bad_words))

### Сохраним подготовленные для МЛ данные в файл preproc.csv

In [15]:
full_df.to_csv('preproc.csv', index=False)